# Deploy a Voice AI bot with Pipecat AI and NIM (and Riva TTS & STT)
In this notebook, we walk through how to craft and deploy a voice AI bot using Pipecat AI. We illustrate the basic Pipecat flow with the `nvidia/llama-3.1-nemotron-70b-instruct` LLM model and Riva for STT (Speech-To-Text) & TTS (Text-To-Speech). However, Pipecat is not opinionated and other models and TTS/STT services can easily be used. See [Pipecat documentation](https://docs.pipecat.ai/server/services/supported-services#supported-services) for other supported services.

Pipecat AI is an open-source framework for building voice and multimodal conversational agents. Pipecat simplifies the complex voice-to-voice AI pipeline, and lets developers build AI capabilities easily and with Open Source, commercial, and custom models. See [Pipecat's Core Concepts](https://docs.pipecat.ai/getting-started/core-concepts) for a deep dive into how it works.

The framework was developed by Daily, a company that has provided real-time video and audio communication infrastructure since 2016. It is fully vendor neutral and is not tightly coupled to Daily's infrastructure.

## Step 1 - Install dependencies
Here we use Daily for transport, OpenAI for context aggregation, Riva for TTS & TTS, and Silero for VAD (Voice Activity Detection). If using different services, for example Cartesia for TTS, one would run `pip install pipecat-ai[cartesia]`.

In [13]:
!pip install pipecat-ai[daily,openai,riva,silero]

I0000 00:00:1734059531.914686  147023 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Step 2 - Configure Daily transport for WebRTC communication
- room_url: Where to connect (and where will navigate to to talk to our bot)
- None: No authentication token needed
- "NVIDIA NIM": The bot's display name
- Enable audio output for text-to-speech playback and enable VAD

In [14]:
# Url to talk to the NVIDIA NIM bot
DAILY_SAMPLE_ROOM_URL="https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test"

from pipecat.audio.vad.silero import SileroVADAnalyzer
from pipecat.transports.services.daily import DailyParams, DailyTransport

transport = DailyTransport(
    DAILY_SAMPLE_ROOM_URL,
    None,
    "NVIDIA NIM",
    DailyParams(
        audio_out_enabled=True,
        vad_enabled=True,
        vad_analyzer=SileroVADAnalyzer(),
        vad_audio_passthrough=True,
    ),
)

2024-12-12 21:12:20.260 | INFO     | pipecat.audio.vad.vad_analyzer:set_params:69 - Setting VAD params to: confidence=0.7 start_secs=0.2 stop_secs=0.8 min_volume=0.6
2024-12-12 21:12:20.260 | INFO     | pipecat.audio.vad.vad_analyzer:set_params:69 - Setting VAD params to: confidence=0.7 start_secs=0.2 stop_secs=0.8 min_volume=0.6
2024-12-12 21:12:20.260 | INFO     | pipecat.audio.vad.vad_analyzer:set_params:69 - Setting VAD params to: confidence=0.7 start_secs=0.2 stop_secs=0.8 min_volume=0.6
2024-12-12 21:12:20.260 | INFO     | pipecat.audio.vad.vad_analyzer:set_params:69 - Setting VAD params to: confidence=0.7 start_secs=0.2 stop_secs=0.8 min_volume=0.6
2024-12-12 21:12:20.264 | DEBUG    | pipecat.audio.vad.silero:__init__:114 - Loading Silero VAD model...
2024-12-12 21:12:20.264 | DEBUG    | pipecat.audio.vad.silero:__init__:114 - Loading Silero VAD model...
2024-12-12 21:12:20.264 | DEBUG    | pipecat.audio.vad.silero:__init__:114 - Loading Silero VAD model...
2024-12-12 21:12:20.2

RuntimeError: unable to select virtual speaker device

## Step 3 - Initialize LLM, STT, and TTS services
We can customize options, for example a different LLM `model` or `voice_id` for FastPitch TTS.

In [15]:
import os
from pipecat.services.nim import NimLLMService
from pipecat.services.riva import FastPitchTTSService, ParakeetSTTService

stt = ParakeetSTTService(api_key=os.getenv("NVIDIA_API_KEY"))

llm = NimLLMService(
    api_key=os.getenv("NVIDIA_API_KEY"), model="meta/llama-3.1-70b-instruct"
)

tts = FastPitchTTSService(api_key=os.getenv("NVIDIA_API_KEY"))

## Step 4 - Define prompt and initialize context aggregator
Edit the prompt as desired.

In [16]:
from pipecat.processors.aggregators.openai_llm_context import OpenAILLMContext

messages = [
    {
        "role": "system",
        "content": "You are a helpful LLM in a WebRTC call. Your goal is to demonstrate your capabilities in a succinct way. Your output will be converted to audio so don't include special characters in your answers. Respond to what the user said in a creative and helpful way that makes a cat pun if it is possible.",
    },
]

context = OpenAILLMContext(messages)
context_aggregator = llm.create_context_aggregator(context)

## Step 5 - Set event handlers
The `on_first_participant_joined` handler tells the bot to start the conversation when you join the call.  
The `on_participant_left` handler sends an EndFrame which signals to terminate the pipeline.

In [17]:
from pipecat.frames.frames import LLMMessagesFrame, EndFrame

@transport.event_handler("on_first_participant_joined")
async def on_first_participant_joined(transport, participant):
    # Kick off the conversation.
    messages.append({"role": "system", "content": "Please introduce yourself to the user and deliver a cat fact."})
    await task.queue_frames([LLMMessagesFrame(messages)])

@transport.event_handler("on_participant_left")
async def on_participant_left(transport, participant, reason):
    print(f"Participant left: {participant}")
    await task.queue_frame(EndFrame())   

## Step 6 - Define the Pipecat pipeline
Here we align the services into a pipeline to process speech into text, send to llm, then turn the llm response text into speech.

In [6]:
from pipecat.pipeline.pipeline import Pipeline
from pipecat.pipeline.runner import PipelineRunner
from pipecat.pipeline.task import PipelineParams, PipelineTask

async def main():
    # Create pipeline to process speech into text, send to llm, then turn the llm response text into speech
    pipeline = Pipeline(
        [
            transport.input(),  # Transport user input
            stt,  # STT
            context_aggregator.user(),  # User responses
            llm,  # LLM
            tts,  # TTS
            transport.output(),  # Transport bot output
            context_aggregator.assistant(),  # Assistant spoken responses
        ]
    )

    # Create PipelineTask
    task = PipelineTask(pipeline, PipelineParams(allow_interruptions=True))

    # Create a pipeline runner to manage the processing pipeline
    runner = PipelineRunner()

    await runner.run(task)

## Step 7 - Run the bot! Then talk to the bot [HERE](https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test)

In [7]:
import sys
import asyncio
import nest_asyncio
nest_asyncio.apply()

from loguru import logger

logger.add(sys.stderr, level="DEBUG")
asyncio.run(main())

2024-12-12 20:46:06.132 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking PipelineSource#1 -> DailyInputTransport#1
2024-12-12 20:46:06.132 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking PipelineSource#1 -> DailyInputTransport#1
2024-12-12 20:46:06.132 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking PipelineSource#1 -> DailyInputTransport#1
2024-12-12 20:46:06.132 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking PipelineSource#1 -> DailyInputTransport#1
2024-12-12 20:46:06.135 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking DailyInputTransport#1 -> ParakeetSTTService#1
2024-12-12 20:46:06.135 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking DailyInputTransport#1 -> ParakeetSTTService#1
2024-12-12 20:46:06.135 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking DailyInputTransport#1 -> ParakeetSTTService#1
2024-12-12 20:46:06.135 | DEBUG    | pipecat.processors.f